__CrossEntropyLoss详解__

In [7]:
import torch
import torch.nn.functional as F

#生成神经网络output，一行对应一个样本，一列对应该样本属于该列类的预测概率
output = torch.randn(4, 5)
print('output:\n',output)

#生成5个样本的标签，并转换为One-Hot编码
target = torch.tensor([2, 1, 3, 4])
print('target:\n',target)
one_hot = F.one_hot(target).float()
print('one_hot:\n',one_hot)

#softmax层运算，每个通道每个像素点的值转换为预测概率值
softmax = torch.exp(output)/torch.sum(torch.exp(output),dim=1).reshape(-1, 1)
print('softmax:\n',softmax)

#对数运算，保证数值稳定性和overflaw、underflaw
logsoftmax = torch.log(softmax)
print('logsoftmax:\n',logsoftmax)

#按照label标号为1的像素点位置读取logsoftmax，计算损失函数值
nllloss = -torch.sum(one_hot*logsoftmax)/target.shape[0]
print('one_hot*logsoftmax:\n',one_hot*logsoftmax)
a = one_hot*logsoftmax
print('a',a.shape)
print('nllloss:\n',nllloss)

#直接使用封装好的CrossEntropyLoss计算损失函数值
loss = torch.nn.CrossEntropyLoss()
value = loss(output, target)
print('loss:\n',value)

output:
 tensor([[-0.5807,  0.3774,  0.6274, -1.1844,  1.1816],
        [-0.3242,  0.9124, -0.4214, -0.3013, -1.3511],
        [-1.2594, -0.4036, -0.3842, -0.5969, -1.3031],
        [ 0.9459,  2.5789, -1.5776,  0.5863, -0.4910]])
target:
 tensor([2, 1, 3, 4])
one_hot:
 tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
softmax:
 tensor([[0.0750, 0.1956, 0.2512, 0.0410, 0.4371],
        [0.1485, 0.5115, 0.1348, 0.1520, 0.0532],
        [0.1156, 0.2721, 0.2774, 0.2242, 0.1107],
        [0.1402, 0.7175, 0.0112, 0.0978, 0.0333]])
logsoftmax:
 tensor([[-2.5897, -1.6316, -1.3816, -3.1935, -0.8275],
        [-1.9069, -0.6704, -2.0042, -1.8840, -2.9338],
        [-2.1575, -1.3017, -1.2823, -1.4950, -2.2012],
        [-1.9650, -0.3320, -4.4885, -2.3246, -3.4019]])
one_hot*logsoftmax:
 tensor([[-0.0000, -0.0000, -1.3816, -0.0000, -0.0000],
        [-0.0000, -0.6704, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.000

In [9]:
import torch
import torch.nn.functional as F
target = torch.randint(0, 5, (4, 4))
print(target)
print(target.shape)
one_hot = F.one_hot(target).float()
print(one_hot.shape)
print(one_hot)

tensor([[1, 2, 2, 0],
        [1, 1, 4, 1],
        [0, 3, 4, 2],
        [1, 4, 0, 4]])
torch.Size([4, 4])
torch.Size([4, 4, 5])
tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.]],

        [[1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0.]],

        [[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.]]])


__咽喉反流CB_Loss__

In [ ]:
import torch
import math
def cb_loss(y_pred, y_true, temp, beta, confusion):
    confusion_ = [torch.sum(temp == i) for i in range(8)]   #统计一张label中每个类别的像素点

    weight = []
    weight_dice = []
    for i, n in zip(confusion_, confusion):
        if i == 0:
            weight.append(0)
            weight_dice.append(1)
        else:
            weight_dice.append(1)
            weight.append(((1.0 - beta) / (1.0 - math.pow(beta, n))))
    
    
    weight = torch.FloatTensor(weight)
    weight_dice = torch.FloatTensor(weight_dice)
    weight = weight.to(y_pred.device)
    weight_dice = weight_dice.to(y_pred.device)


    criterion_train = torch.nn.CrossEntropyLoss(weight=weight)

    loss = criterion_train(y_pred.float(), temp.float())

    return loss

__CB_Loss__

In [ ]:
import math
import numpy as np

def cb_loss(y_pred, y_true, temp, beta, confusion):
    
    
    #统计标签MASK中每个类的样本数量，temp=y_true
    confusion_ = [torch.sum(temp == i) for i in range(2)]

    weight = []
    weight_dice = []
    for i, n in zip(confusion_, confusion):
        if i == 0:
            weight.append(0)
            weight_dice.append(1)
        else:
            weight_dice.append(1)
            weight.append(((1.0 - beta) / (1.0 - math.pow(beta, n))))
    
    weight = torch.FloatTensor(weight)
    weight_dice = torch.FloatTensor(weight_dice)
    weight = weight.to(y_pred.device)
    weight_dice = weight_dice.to(y_pred.device)


    criterion_train = torch.nn.CrossEntropyLoss(weight=weight)

    loss = criterion_train(y_pred.float(), temp.float())

    return loss




In [7]:
import torch

target = torch.randint(0, 5, (5, 5))
print(target)
confusion_ = [torch.sum(target == i) for i in range(5)]
print(confusion_)

tensor([[2, 3, 3, 1, 2],
        [0, 2, 1, 3, 3],
        [1, 2, 1, 1, 2],
        [2, 0, 3, 0, 0],
        [1, 0, 3, 1, 4]])
[tensor(5), tensor(7), tensor(6), tensor(6), tensor(1)]


__数据集加载最新版本（negative:positive=7:3）（代码保留）__

In [18]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class Generate_Dataset1(Dataset):
    def __init__(self, path):
        self.path = path
        self.data_path = glob.glob\
            (os.path.join(path,'negative_input/*.npy'))  #读取data文件夹下所有.npy格式文件

    def __getitem__(self, index):
        data_path = self.data_path[index]
        # print(data_path)
        data = np.load(data_path)      #读取输入数据
        tensor_data = torch.from_numpy(data)
        
        label_path = data_path.replace('input/negative_input', 'label/negative_label')
        label = np.load(label_path)    #读取标签数据
        tensor_label = torch.from_numpy(label)

        return tensor_data, tensor_label

    def __len__(self):
        return len(self.data_path)

class Generate_Dataset2(Dataset):
    def __init__(self, path):
        self.path = path
        self.data_path = glob.glob\
            (os.path.join(path,'positive_input/*.npy'))  #读取data文件夹下所有.npy格式文件

    def __getitem__(self, index):
        data_path = self.data_path[index]
        # print(data_path)
        data = np.load(data_path)      #读取输入数据
        tensor_data = torch.from_numpy(data)
        
        label_path = data_path.replace('input/positive_input', 'label/positive_label')
        label = np.load(label_path)    #读取标签数据
        tensor_label = torch.from_numpy(label)

        return tensor_data, tensor_label

    def __len__(self):
        return len(self.data_path)

#测试载入数据程序
if __name__ == '__main__':
    #设置dataloader固定生成器
    generator = torch.Generator().manual_seed(42)
 
    #读取并载入negative_dataset
    print('读入negative_sample...')
    print('***********************************************')
    dataset1 = Generate_Dataset1('../Data/dataset(positive_negative)/input/')
    negative_train_size = int(len(dataset1) * 0.7)
    print('negative_train_size:', negative_train_size)
    negative_validate_size = int(len(dataset1) - negative_train_size)
    print('negative_validate_size:', negative_validate_size)

    negative_train_dataset, negative_validate_dataset = torch.utils.data\
                .random_split(dataset1, [negative_train_size, negative_validate_size], generator = generator)

    print("读入数据个数为：", len(negative_train_dataset)+len(negative_validate_dataset))
    print('***********************************************')
    print('读入完毕！')
    print('\n')
    # train_loader = DataLoader(negative_train_dataset, batch_size=1, shuffle=True)
    # validate_loader = DataLoader(negative_validate_dataset, batch_size=1, shuffle=True)
    
    # t = 0
    # for negative_train, negative_label in train_loader:
    #     t += 1
    #     print(negative_train.shape)
    #     print(negative_label.shape)
    # print('共有',t,'个训练集')
    
    # n = 0
    # for negative_validate, negative_label in validate_loader:
    #     n += 1
    #     print(negative_validate.shape)
    #     print(negative_label.shape)
    # print('共有',n,'个训练集')







    #读取并载入positive_dataset
    print('读入positive_sample...')
    print('***********************************************')
    
    dataset2 = Generate_Dataset2('../Data/dataset(positive_negative)/input/')
    positive_train_size = int(len(dataset2) * 0.7) + 1
    print('positive_train_size:',positive_train_size)
    positive_validate_size = int(len(dataset2) - positive_train_size)
    print('positive_validate_size:', positive_validate_size)
    positive_train_dataset, positive_validate_dataset = torch.utils.data\
                .random_split(dataset2, [positive_train_size, positive_validate_size], generator = generator)
    
    
    
    print('positive_train_dataset:',np.array(*positive_train_dataset.indices).shape)




    print("读入数据个数为：", len(positive_train_dataset)+len(positive_validate_dataset))
    print('***********************************************')
    print('读入完毕！')
    # train_loader = DataLoader(positive_train_dataset, batch_size=1, shuffle=True)
    # validate_loader = DataLoader(positive_validate_dataset, batch_size=1, shuffle=True)
    
    
    
    # t = 0
    # for positive_train, positive_label in train_loader:
    #     t += 1
    #     # print(positive_train.shape)
    #     # print(positive_label.shape)
    # # print('共有',t,'个训练集')
    
    # n = 0
    # for positive_validate, positive_label in validate_loader:
    #     n += 1
    #     # print(positive_validate.shape)
    #     # print(positive_label.shape)
    # # print('共有',n,'个训练集')

    train_dataset = torch.utils.data.ConcatDataset([negative_train_dataset, positive_train_dataset])
    validate_dataset = torch.utils.data.ConcatDataset([negative_validate_dataset, positive_validate_dataset])
    print('\n')
    print('train_dataset/validate_dataset数量统计...')
    print('***********************************************')
    print('Number of train_dataset:',len(train_dataset))
    print('Number of validate_dataset:',len(validate_dataset))
    print('***********************************************')
    # print(len(train_dataset))
    # print(len(validate_dataset))
    # print(list(train_dataset))
    # m = 0
    # for data, label in validate_dataset:
    #     m += 1
    #     # print(data)
    #     print('******************************************************************')
    #     # print(label)
    #     print('一共',m,'个数据')

    print('\n')
    print('开始加载正式数据集...')
    print('***********************************************')
    traindata_Loader = DataLoader(train_dataset, batch_size = 50, shuffle=True)
    validatedata_Loader = DataLoader(validate_dataset, batch_size = 50, shuffle=True)
    q = 0
    for train, label in traindata_Loader:
        # print(train.shape)
        # print(label.shape)
        q += 1
    print('测试集一共划分为：',q,'个批次')
    p = 0
    for validate, label in validatedata_Loader:
        # print(validate.shape)
        # print(label.shape)
        p += 1
    print('验证集一共划分为：',p,'个批次')
    print('***********************************************')
    print('正式数据集加载完毕！')

读入negative_sample...
***********************************************
negative_train_size: 6733
negative_validate_size: 2886
读入数据个数为： 9619
***********************************************
读入完毕！


读入positive_sample...
***********************************************
positive_train_size: 222
positive_validate_size: 95


TypeError: array() takes from 1 to 2 positional arguments but 222 were given